## LangChain과 ChatGPT

이 노트북은 LangChain에서 지원되는 ChatGPT 및 관련 기능에 대한 간략한 소개를 제공합니다.

시작하기 전에 이러한 라이브러리를 설치하세요. 이를 위한 전용 환경을 만드는 것이 가장 이상적입니다.

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv

In [2]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv
load_dotenv()

True

환경 변수를 로드합니다. 원하는 환경 변수는 무엇이든 사용할 수 있지만 저는 `python-dotenv`를 사용했습니다. OPENAI_API_KEY`로 `.env` 파일을 생성한 다음 다음과 같이 로드합니다:

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

여기](https://langchain.readthedocs.io/en/latest/modules/chat/getting_started.html)의 코드를 적용하고 있습니다.

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [5]:
# chat mode instance
chat = ChatOpenAI(temperature=0)

/Users/elvissaravia/opt/miniconda3/envs/peguide/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


ChatGPT는 역할에 따라 식별 가능한 다양한 유형의 메시지를 지원합니다. LangChain. 오픈아이`를 사용하여 ChatGPT에 기본 호출을 하는 방법을 기억하시나요? 다음은 그 예시입니다:

```python
MODEL = "gpt-3.5-turbo"

응답 = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "시스템", "콘텐츠": "귀하는 AI 연구 조교입니다. 기술적이고 과학적인 어조를 사용합니다."},
        {"role": "사용자", "콘텐츠": "안녕하세요, 누구세요?"},
        {"role": "어시스턴트", "콘텐츠": "안녕하세요! 저는 AI 연구 조교입니다. 오늘은 무엇을 도와드릴까요?"},
        {"role": "사용자", "콘텐츠": "블랙홀의 생성에 대해 알려주실 수 있나요?"}
    ],
    temperature=0,
)
```

LangChain은 임의의 역할 매개변수(`ChatMessage`)를 포함하여 이러한 다양한 유형의 메시지를 지원합니다. 시도해 봅시다:

In [7]:
USER_INPUT = "I love programming."
FINAL_PROMPT = """Classify the text into neutral, negative or positive. 

Text: {user_input}. 
Sentiment:"""

chat.invoke([HumanMessage(content=FINAL_PROMPT.format(user_input=USER_INPUT))])

AIMessage(content='positive')

시스템 명령과 사용자가 제공한 작업이 포함된 예제를 살펴보겠습니다.

In [8]:
messages = [
    SystemMessage(content="You are a helpful assistant that can classify the sentiment of input texts. The labels you can use are positive, negative and neutral."),
    HumanMessage(content="Classify the following sentence: I am doing brilliant today!"),
]

chat.invoke(messages)

AIMessage(content='The sentence "I am doing brilliant today!" can be classified as positive sentiment.')

이제 인간과 AI 연구 조교 간의 교환을 포함하는 다른 예를 살펴보겠습니다:

In [9]:
messages = [
    SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
    HumanMessage(content="Hello, who are you?"),
    AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
    HumanMessage(content="Can you tell me about the creation of black holes?")
]

chat.invoke(messages)

AIMessage(content="Certainly! Black holes are fascinating astronomical objects that form from the remnants of massive stars. The creation of a black hole occurs through a process known as stellar collapse.\n\nWhen a massive star exhausts its nuclear fuel, it can no longer sustain the outward pressure generated by nuclear fusion. As a result, the star's core collapses under the force of gravity. This collapse is triggered by the imbalance between the inward gravitational force and the outward pressure.\n\nDuring the collapse, the star's core becomes incredibly dense, packing an enormous amount of mass into a tiny volume. This extreme density leads to the formation of a singularity, a point of infinite density at the center of the black hole.\n\nSurrounding the singularity is the event horizon, which is the boundary beyond which nothing, not even light, can escape the gravitational pull of the black hole. The event horizon is determined by the mass of the black hole, with larger black ho

이러한 요청을 일괄 처리하여 응답을 생성하는 기능도 있습니다(`chat.response()` 사용):

In [10]:
batch_messages = [
    [
        SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
        HumanMessage(content="Hello, who are you?"),
        AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
        HumanMessage(content="Can you tell me about the creation of black holes?")
    ],
    [
        SystemMessage(content="You are an AI research assistant. You use a tone that is technical and scientific."),
        HumanMessage(content="Hello, who are you?"),
        AIMessage(content="Greeting! I am an AI research assistant. How can I help you today?"),
        HumanMessage(content="Can you explain the dark matter?")
    ]
]

chat.generate(batch_messages)

LLMResult(generations=[[ChatGeneration(text="Certainly! Black holes are fascinating astronomical objects that are formed through the gravitational collapse of massive stars. The creation of a black hole occurs when a star exhausts its nuclear fuel and can no longer sustain the outward pressure generated by nuclear fusion in its core.\n\nWhen a massive star reaches the end of its life, it undergoes a supernova explosion. During this explosive event, the outer layers of the star are ejected into space, while the core collapses inward due to its own gravity. If the core's mass is above a certain threshold, known as the Tolman-Oppenheimer-Volkoff (TOV) limit, the collapse continues until it forms a black hole.\n\nThe core collapse is an incredibly dense and energetic process. As the core collapses, its mass becomes concentrated into an extremely small volume, resulting in a gravitational field so strong that not even light can escape from it. This region of space is known as the event hori

위의 예시를 보면 프롬프트 템플릿을 사용하는 것이 더 쉬울 수 있습니다. LangChain도 지원합니다. 아래에서 시도해 보겠습니다:

In [11]:
template = "You are a helpful assistant that can classify the sentiment of input texts. The labels you can use are {sentiment_labels}. Classify the following sentence:"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{user_input}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [12]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


chat.invoke(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="I am doing brilliant today!").to_messages())

AIMessage(content='positive')

In [13]:
chat.invoke(chat_prompt.format_prompt(sentiment_labels="positive, negative, and neutral", user_input="Not sure what the weather is like today.").to_messages())

AIMessage(content='neutral')